In [293]:
from pathlib import Path
import re
import pandas as pd
import openpyxl
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from jupyter_contrib_nbextensions.application import main

#read
data_folder = Path("C:/Users/Jing Li/Desktop/cv2db/project/Inputs")
file = data_folder / "Echo Roster.xlsx"

In [294]:
data = pd.ExcelFile(file)
df_pro = data.parse("Programmatic Consultants")
df_fis = data.parse("Fiscal Consultants")
df_fis_b = data.parse("Back-up Fiscal Consultants")   

Extraction from now on

In [295]:
# Extract Expertises
expertises = {'Coaching & Training': ["coach"],
              'Federal government agency': ["federal"],
              'Meeting Facilitation': ["meeting"],
              'Report Writing': ["report", "proposal", "write", "wrote"],
              'Budget and fiscal management': ["budget", "fiscal manage",
                                               "fiscal consul", "fiscal oversight"],
              'Epidemiology': ['Epidemiology'],
              'Global health': ["global health"],
              'Healthcare policy and regulations': ["policy", "regulation"],
              'Infectious Diseases': ["Infectious"],
              'International Relations': ["international relation"],
              "Behavioral Science": ["behavior"],
              'Mental Health': ['mental health'],
              'Psychiatry': ["Psychiatry"],
              'Substance abuse': ["substance abuse"],
              'Human Trafficking': ["trafficking"],
              'Nutrition/Food security': ['food', 'nutrition'],
              'Hepatitis C Virus (HCV)': ['HCV'],
              'State procurement systems & program income reporting': ['state procurement', 'program income'],
              'State Compliance Monitoring': ['Compliance'],
              'Capacity building': ["capacity building", "capacity develop",
                                    "building their capacity", "build their capacity",
                                   "build capacity", "services capacity", "system capacity",
                                   "capacity expansion", "built capacity"],
              'Clinic Operations': ["clinic"],
              'Financial analysis & management': ["financial", "finance", "accounting"],
              'Fund development & Sustainability': ["fund ", "funds", "funding"],
              'Grants management': ["grants"],
              'Human Capital Systems': ["Human Capital"],
              'Human resources management': ["Human resource"],
              'Humility and Organizational Leaders': ['Humility'],
              'Organizational Development&Transformation': ['Organizational Development',
                                                           'Transformation'],
              'Quality improvement and management': ["quality program", "quality improve",
                                                     "improve quality"],
              'State Procurement System': ["Procurement"],
              'Strategic planning': ["Strategic"],
              'Auditing Federally-funded Grants Program': ["Federally-funded Grant"],
              'Federal Acquisition Regulation (FAR)': [r"\bFAR"],
              'Data collection, analysis and reporting': ["data collect", "data analy"],
              'Family Planning and Reproductive Health': ["family planning", "FP/RH", "reproductive"],
              'Program evaluation & analysis': ["evaluation", "evalute", "evaluator"],
              'Programmatic Assessment & Management': ["Programmatic"],
              'Time-constrainted Financial Analysis': ["Time-constrainted", 'Time constrainted'],
              'Nurse/Administration': ['Nurse/Administration']
             }

In [296]:
#Extract Federal Project
fp = {'Rural Health (FORHP)': ['FORHP'],
      'Health Centers (BPHC)': ['BPHC'],
      'HIV/AIDS Domestic (HAB Ryan White)': ['Ryan White'],
      'HIV/AIDS International (HAB PEPFAR)': ['PEPFAR'],
      'Maternal and Child Heath (MCHB, ACF)': ['MCHB', "maternal and child"],
      'Maternal, Infant, and Early Childhood Home Visiting Program (MIECHV)': ['MIECHV'],
      'Population Assessment of Tobacco and Health Study (PATH)': ['PATH '],
      'Global Trade Analysis Project (GTAP)': ['GTAP'],
      'The Joint United Nations Programme on HIV/AIDS (UNAIDS)': ['UNAIDS'],
      'Nursing Education Partnership Initiative (NEPI)': ['NEPI'],
      'Head Start': ['Head Start'],
      'Early Head Start': ['Early Head Start', 'EHS'],
      'Community Services Block Grant (CSBG)': ['CSBG'],
      'The HIV/AIDS Bureau (HAB)': ['HAB'],
      'Environmental Protection Agency (EPA)': ['EPA'],
      'General Services Administration (GSA)': ['GSA'],
      'Department of Energy (DOE)': ['DOE'],
      'Government Accountability Office (GAO)': ['GAO'],
      'Statement of Budgetary Resources (SBR)': ['SBR'],
      'Enterprise Resource Programs (ERP)': ['ERP'],
      'Strengthening Accountabilityin the Glocal Economy (SAGE) Fund': ['SAGE'],
      'Head Start Enterprise System': ['Head Start Enterprise System'],
      'Temporary Assistance for Needy Families (TANF)': ['TANF'],
      'Community Health Center (CHC)': ['CHC'],
      'Federal Highway programs': ['Highway pro'],
      'Corporation for National and Community Service (CNCS)': ['CNCS'],
      'Uniform Guidance (2 CFR Part 200)': ['2 CFR Part 200', '2 CFR', ' 2CFR Part 200'],
      'Uniform Guidance (45 CFR Part 75)': ['45CFR75'],
      'Deep Foundations Institute (DFI)': ['DFI'],
      'OMB A-133 Compliance Supplement': ['A-133'],
      'Linking Actions for Unmet Needs in Children’s Health (LAUNCH)': ['LAUNCH'],
      'Healthy Start Initiative': ['Healthy Start'],
      'Home Visiting Collaborative Improvement and Innovation Network (HV CoIIN)': ['CoIIN'],
      'Parents as Teachers (PAT)': ['Parents as Teachers (PAT)'],
      'Race to the Top': ['Race to the Top'],
      'Home Instruction for Parents of Preschool Youngsters (HIPPY)': ['HIPPY'],
      'Healthy Families America (HFA)': ['HFA'],
      'Nurse-Family Partnership (NFP)': ['NFP'],
      'Evidence-based Home Visiting': ['Evidence-based'],
      'Center on the Social and Emotional Foundations for Early Learning (CSEFEL)': ['CSEFEL']
     }

In [306]:
# certificate
certificates = {'Certified Registered Nurse Practitioner (CRNP)': ["N\.P\.", "Nurse Practitioner"],
                'Mortgage Credit Certificate (MCC)': ["MCC"],
                'Board Certified Coach (BCC)': ["BCC"],
                'Certified Public Accountant (CPA)': ["CPA", "Certified Public Accountant"],
                'Cardiopulmonary Resuscitation (CPR)': ["CPR"],
                'Certified Valuation Analyst (CVA)': ["CVA"],
                'Advanced Cardiovascular Life Support (ACLS)': ["ACLS"],
                'Chartered Global Management Accountant (CGMA)': ["CGMA"],
                'Physician Assistant (PA) from NCCPA': [" PA ", "Physician Assistant"],
                'Project Management Professional (PMP)': ["PMP"],
               }

In [307]:
# target_pop

population = {'Early Childhood': ["early child"],
              'Elderly population': ["elder", "senior health"],
              'Orphans': ["Orphan"],
              'Malaria': ["Malaria"],
              'Female population': ["female", "girls", "women"],
              'HIV/AIDS population': ["hiv", "aids"],
              'Homeless population': ["homeless"],
              'Infants': ["infants", "baby"],
              'LGBTQ': ["lgbt", "transgender"],
              'Low-income population': ["low-income", "poor", "poverty"],
              'Maternal and Children': ["mother", "maternal", "child"],
              'People with disability': ["disable", "disabilit"],
              'Rural Health': ["rural"],
              'Youth': ["youth", "adolescent", "teen"],
              'Tribal Home Visiting': ["tribal"],
             }


In [308]:
#construct Check functions
def Check(data, dic): # data is a list/tuple of strings of ONE person
    p = []
    p_data = " ".join(data) #p_data is a string
    for key in dic:
        for kw in dic[key]:
            if re.search(kw, p_data, re.IGNORECASE):
                p.append(key)
                break
    return p

# construct new dataframe
def DF_new(df_old):
    df_old.fillna('',inplace=True)
    Expertises = [Check(row[1][1:], expertises) for row in df_old.iterrows()]
    FP = [Check(row[1][1:], fp) for row in df_old.iterrows()]
    Certificates = [Check(row[1][1:], certificates) for row in df_old.iterrows()]
    Population = [Check(row[1][1:], population) for row in df_old.iterrows()]
    df_new = pd.DataFrame({"id": df_old["id"], "Federal Project": FP, "Expertises": Expertises,
                              "Certificates": Certificates, "Target Population": Population})
    return df_new

df = pd.concat([DF_new(df_pro), DF_new(df_fis), DF_new(df_fis_b)])

In [309]:
# Normalization
def normalization(col_name):
    output = []
    for i in df["id"]:
        for j in df.loc[df['id'] == i, col_name]:
            for k in j:
                output.append([i, k])
    return output

wb = Workbook()
interested_cols = ["Federal Project", "Expertises", "Certificates", "Target Population"]

for i in interested_cols:
    ws = wb.create_sheet(title=i[:7])
    result = normalization(i)
    row = 1
    for j in result: #each j is a [people, skill] list
        ws["A"+str(row)] = j[0]
        ws["B"+str(row)] = j[1]
        row += 1
    
wb.save(data_folder / "atomized Echo.xlsx")